In [1]:
from pathlib import Path
import cv2
import sys
import numpy as np
from skimage import feature, measure
import livecell_tracker
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset
import livecell_tracker.segment
from livecell_tracker import core
import livecell_tracker.core.utils
from tqdm import tqdm
import json
from livecell_tracker.core import (
    SingleCellTrajectory,
    SingleCellStatic,
    SingleCellTrajectoryCollection,
)
import json

seg_dir_path = r"""../datasets/test_data/track_sort/seg_output_3_imgs_day0_group1"""
label_seg_imgs = core.datasets.LiveCellImageDataset(dir_path=seg_dir_path, ext="png")

label_seg_imgs = LiveCellImageDataset(dir_path=seg_dir_path, ext="png", num_imgs=3)
dataset_dir_path = Path(
    r"../cxa-data/june_2022_data_8bit_png/restart_day0_Group 1_wellA1_RI_MIP_stitched"
)

raw_imgs = LiveCellImageDataset(dir_path=dataset_dir_path, ext="png", num_imgs=3)
segmentation_result_json_path = r"../datasets/test_data/track_sort/seg_output_3_imgs_day0_group1/segmentation_results.json"
segmentation_results = json.load(open(segmentation_result_json_path, "r"))
segmentation_results;

6 png img file paths loaded: 
6 png img file paths loaded: 
288 png img file paths loaded: 


In [2]:
raw_imgs[1]

array([[14, 14, 16, ..., 13, 14, 14],
       [15, 14, 17, ..., 13, 13, 14],
       [15, 14, 14, ..., 14, 14, 14],
       ...,
       [14, 13, 13, ..., 11, 12, 13],
       [14, 13, 13, ..., 12, 12, 13],
       [14, 14, 14, ..., 12, 11, 12]], dtype=uint8)

In [3]:
import napari
from skimage import data
# viewer = napari.view_image(raw_imgs[0], name='image')
viewer = napari.view_image(raw_imgs.to_dask(), name='image', cache=True)
# all_imgs = LiveCellImageDataset(dataset_dir_path, ext="png", num_imgs=none)
# viewer = napari.view_image(all_imgs.to_dask(), name='all-image', cache=False)

In [4]:
traj_collection_json_path = "../datasets/test_data/traj_analysis/track_singleCellTrajectoryCollection.json"
traj_collection_json = json.load(open(traj_collection_json_path, "r"))
trajectory_collection = SingleCellTrajectoryCollection().load_from_json_dict(traj_collection_json)

In [5]:
from livecell_tracker.core.napari_visualizer import NapariVisualizer
shape_layer = NapariVisualizer.viz_traj_collection(trajectory_collection, viewer)


In [6]:
shape_layer.data[0]

array([[   0. ,   57.5, 1259.5],
       [   0. ,   57.5, 1323.5],
       [   0. ,  150.5, 1323.5],
       [   0. ,  150.5, 1259.5]])

In [7]:
def set_shapes_features(shapes_layer, step2shapes, step):
    # step is a 4D coordinate with the current slider position for each dim
    column = step[0]  # grab the leading ("time") coordinate
    new_data = []
    print(step)
    if step in step2shapes:
        new_data = step2shapes[step]
    shapes_layer.data = new_data
    shapes_layer.face_color = 'value'  # force features refresh


# viewer.dims.events.current_step.connect(
#         lambda event: set_shapes_features(shapes_layer, traj_1.get_sc_napari_shapes(), event.value)
#         )
